# Introducción a Keras y TensorFlow

TensorFlow (TF) es una plataforma basada en Python, gratuita y de código abierto, desarrollada principalmente por Google. Similar a NumPy, el propósito principal de TensorFlow es permitir al usuario manipular expresiones matemáticas en tensores numéricos.

Keras es una API (Application Programming Interface) para Python, construida sobre TensorFlow, que provee una manera conveniente para definir y entrenar cualquier tipo de modelo de DL. 

## Tensores constantes y variables

Para hacer algo en TF, necesitamos algunos tensores. Los tensores deben crearse con algún valor inicial. Por ejemplo, podríamos crear tensores que sólo contengan unos o ceros, o tensores que tengan valores de alguna distribución aleatoria.

In [ ]:
import tensorflow as tf

a = tf.ones(shape = (2,1))  # Equivalente a np.ones(shape=(2,1))

print(a)

In [ ]:
b = tf.zeros(shape = (3,3)) # Equivalente a np.zeros(shape=(2,1))

print(b)

In [ ]:
# Creamos un tensor con valores aleatorios a partir de una distribución
# normal con media = 0 y desviación estándar = 1

c = tf.random.normal(shape = (3,1), mean = 0.0, stddev = 1.0)

print(c)

In [ ]:
d = tf.random.uniform(shape = (3,1), minval = 0., maxval = 1.)

print(d)

Para entrenar un modelo, necesitamos actualizar su estado, que es un conjunto de tensores. Como los tensores no son asignables, tenemos que utilizar *variables* con la clase `tf.Variable`, que nos permite modificar el estado en TF. 

Para crear una variable, necesitamos dar un valor inicial, como un tensor aleatorio.

In [ ]:
v = tf.Variable(initial_value = tf.random.normal(shape = (3,1)))

print(v)

El estado de la variable se puede modificar a través del método `assign`, como sigue:

In [ ]:
v.assign(tf.ones((3,1)))

También funciona para un subconjunto de entradas.

In [ ]:
v[0,0].assign(3.)

### Operaciones tensoriales

TF ofrece un conjunto de operaciones tensoriales para expresar fórmulas matemáticas. Por ejemplo, si queremos obtener el cuadrado de las entradas de un tensor, usamos

In [ ]:
print(d)

In [ ]:
d_sq = tf.square(d)

d_sq

Para sacar la raíz cuadrada

In [ ]:
d_sqrt = tf.sqrt(d)

d_sqrt

Sumamos dos tensores, entrada a entrada

In [ ]:
suma = c + d

suma

### Gradient Tape

`GradientTape` es una API que nos permite utilizar las capacidades de diferenciación automática de TF. Es un esquema de Python que va a "registrar" las operaciones tensoriales que corran dentro de él, en forma de un grafo computacional (a veces llamado *tape*). Ese grafo puede usarse para recuperar el gradiente de cualquier salida con respecto a cualquier variable o conjunto de variables (las instancias de la clase `tf.Variable`). 

Una `tf.Variable` es un tipo específico de tensor que debe mantener un estado mutable, por ejemplo, los pesos de una red neuronal siempre son instancias del tipo `tf.Variable`.

In [ ]:
x = tf.Variable(0.) # Variable escalar con valor inicial de 0

with tf.GradientTape() as tape: # Iniciamos el esquema de GradientTape
    y = 2 * x + 3               # Dentro del esquema, aplicamos operaciones tensoriales a la variable

dy_dx = tape.gradient(y, x) # Utilizamos la tape para recuperar el gradiente de la salida y respecto a x

print(dy_dx)

Veamos otro ejemplo, ahora con un tensor un poco más complejo.

In [ ]:
x = tf.Variable(tf.random.uniform((2,2))) # Variable de forma (2,2) con valores iniciales todos ceros

with tf.GradientTape() as tape:
    y = 2 * x + 3

grad_of_y_wrt_x = tape.gradient(y, x)

grad_of_y_wrt_x



## Ejemplo: Clasificador lineal en TensorFlow

Ya vimos tensores, variables y operaciones tensoriales, y ya sabemos cómo calcular gradientes. Podemos construir cualquier modelo de ML que se base en el método del gradiente descendente. Veamos.

Supongamos que nos piden implementar un clasificador lineal usando TF. Vamos a ir paso a paso, así que vamos a generar datos sintéticos bonitos y separables para trabajar: dos clases de puntos en un plano.

Generaremos cada clase de puntos dibujando sus coordenadas a partir de una distribución aleatoria con matriz de covarianza y media específicas. Recordemos que, de forma intuitiva, la matriz de covarianza describe la forma de la nube de puntos, y que la media describe su posición en el plano. 

Para este ejemplo vamos a utilizar la misma matriz de covarianza para cada una de las nubes de puntos, pero utilizaremos dos medias distintas, por lo que las nubes van a tener la misma forma, pero posiciones distintas.

Primero definiremos el número de puntos por clase que queremos, que van a ser, 1000 en cada nube.

Después, generamos las muestras "negativas" de 1000 puntos aleatorios. La matriz de covarianza coresponde a una nubecita en forma de óvalo orientada desde abajo a la izquierda hacia arriba y a la derecha.

El otro conjunto de datos es lo mismo, pero con media distinta.

In [ ]:
num_samples_per_class = 1000


negative_samples = np.random.multivariate_normal(
                                                mean = [0, 3],
                                                cov = [[1, 0.5], [0.5, 1]],
                                                size = num_samples_per_class)

positive_samples = np.random.multivariate_normal(
                                                mean = [3, 0],
                                                cov = [[1, 0.5], [0.5,1]],
                                                size = num_samples_per_class)

En la celda anterior, las variables `negative_samples` y `positive_samples` son arrays de forma (1000,2). Pongamos uno sobre otro para formar un sólo array de (2000,2).

In [ ]:
inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)

Ahora, tenemos que generar también las correspondientes etiquetas objetivo, que en este caso serán un array de ceros y unos de forma (2000,1), donde `targets[i,0]` va a ser 0 si `inputs[i]` pertenece a la clase 0 (y visceversa).

In [ ]:
targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype = 'float32'),
                    np.ones((num_samples_per_class, 1), dtype = 'float32')))

Grafiquemos los datos con Matplotlib.

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(inputs[:,0], inputs[:,1], c = targets[:,0])

plt.show()

Bien, tenemos los datos de entrada listos. Ahora crearemos un clasificador lineal que aprenda a separar estos dos aglomerados. Un clasificador lineal es una transformación afín (`predicción = W • entrada + b`) entrenado para minimizar el cuadrado de la diferencia entre las predicciones y los objetivos.

Creamos entonces nuestras variables `w` y `b`, inicializando con valores aleatorios y con ceros, respectivamente.

In [ ]:
input_dim = 2 # Las entradas son puntos en 2 dimensiones

output_dim = 1 # Mientras que la salida será 0 o 1

w = tf.Variable(initial_value = tf.random.uniform(shape = (input_dim, output_dim)))

b = tf.Variable(initial_value = tf.zeros(shape = (output_dim)))


Definimos la función de paso:

In [ ]:
def model(inputs):
    return tf.matmul(inputs, w) + b

Como nuestro clasificador lineal opera en entradas de 2 dimensiones, `w` en realidad está formado por dos coeficientes escalares, `w1` y `w2`: `w = [[w1], [w2]]`.

Mientras que `b` es un solo coeficiente escalar. Por lo que, para cada dato de entrada `[x,y]`, su predicción será: `prediction = [[w1]. [w2]] • [x,y] + b = w1 * x + w2 * y + b`.

Ahora, definimos la función de costo, que en este caso corresponde al error cuadrático medio.

In [ ]:
def square_loss(targets, predictions):
    per_sample_losses = tf.square(targets - predictions)
    return tf.reduce_mean(per_sample_losses)

`per_sample_losses` será un tensor con la misma forma que los objetivos y las predicciones y que va a contener los scores de la función de costo.

El método `tf.reduce_mean()` promedia todos los valores de la función de costo y nos da un sólo número.

Ahora sigue el paso de entrenamiento, donde se va a recibir un poco de datos de entrenamiento y se van a actualizar los pesos `w` y el valor `b` para reducir el costo sobre los datos.

In [ ]:
learning_rate = 0.1

def training_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = square_loss(predictions, targets)
    grad_loss_wrt_w, grad_loss_wrt_b = tape.gradient(loss, [w,b])
    w.assign_sub(grad_loss_wrt_w * learning_rate)
    b.assign_sub(grad_loss_wrt_b * learning_rate)
    return loss

Por simplicidad, se hará *batch training* en vez de *mini-batch training*: vamos a correr cada paso de entrenamiento (calcular el gradiente y actualizar los pesos) para todos los datos, en vez de iterar en pequeños conjuntos del conjunto completo. 

In [ ]:
for step in range(40):
    loss = training_step(inputs, targets)
    print(f'Loss at step {step}: {loss:.4f}')

Después de 40 iteraciones, el costo del entrenamiento parece estabilizarse alrededor de 0.0255. Grafiquemos cómo nuestro modelo lineal clasifica los datos de entrenamiento.

Como nuestros objetivos son ceros y unos, una entrada dada puede ser clasificada como "0", si su valor de predicción es menor a 0.5, y como "1" si está por arriba de 0.5


In [ ]:
predictions = model(inputs)

plt.scatter(inputs[:,0], inputs[:,1], c = predictions[:,0] > 0.5)
plt.show()

Casi no se nota la diferencia entre las predicciones del modelo (gráfica de arriba) y los datos de entrada. 

Recordemos que el valor de la predicción para cualquier punto $(x,y)$ es simplemente:

$$ \hat{y} = w_1 x + w_2 y + b  $$

Y las clases a las que se clasifica la predicción son tales que:

$$ \hat{y} = \begin{cases}
            0 & \text{si} & w_1 x + w_2 y + b < 0.5 \\
            1 & \text{si} & w_1 x + w_2 y + b > 0.5 \\

\end{cases} $$

Pero pues esto equivale a la ecuación de una recta:

$$  w_1 x + w_2 y + b = 0.5 \quad \to \quad y = - \frac{w_1}{w_2} x + \frac{0.5 - b}{w_2}$$

Arriba de la línea está la clase 1 y por debajo la clase 0. 

Grafiquemos.

In [ ]:
x = np.linspace(-1, 4, 100)

y = -w[0] / w[1] * x + (0.5 - b) / w[1]

plt.plot(x,y, "-r")
plt.scatter(inputs[:,0], inputs[:,1], c = predictions[:,0] > 0.5)
plt.show()


De esto se trata la tarea de un clasificador. Encuentra los parámetros de una línea (o, en muchas dimensiones, un hiperplano) que logre separar adecuadamente dos tipos de datos.